### Check an endpoint
This notebook aims to run the pipeline on a given endpoint to check to see if it will be successful. This includes collecting, pipeline and datset stages. It aims to also highlight useful information as a summary as to whether the endpoint would be successful on our platform. it will download all relevant data to do this and hence might be disk intensive. You'll need to provide the following information:
- collection - this is the collection that the dataset belongs to, this can be extracted from the specification but for this notebook we ask to provide it incase you want to test the pipeline on something which isn't being included in the main site right now
- dataset - this is the dataset that the endpoint is meant to provide data for, technically this can be multiple datasets but this this use case it should just be one. It is also the name of the pipeline that is ran on the individual resources that are downloaded from the endpoint. the terms dataset/pipline are often the same
- organisation - the organisation identifier to be used for the endpoint
- endpoint url - the actual url needed for the endpoint
- plugin - often we use plugins to download the data this is only needed for specific endpoints

In [1]:
import os
import pandas as pd
from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from digital_land.collection import Collection

In [2]:
collection_name = 'article-4-direction-collection'
dataset = 'article-4-direction-area'
organisation = 'local-authority-eng:NET'
endpoint_url = 'https://datamillnorth.org/download/newcastle-article-4-directions/4c650010-8d88-4bba-9f4a-85587742a64d/article4directionareas.csv'
plugin = None
additonal_column_mappings=None
data_dir = '../data/endpoint_checker'

In [3]:
run_endpoint_workflow(
    collection_name,
    dataset,
    organisation,
    endpoint_url,
    plugin,
    data_dir
)

HTTP Error 404: Not Found
HTTP Error 404: Not Found
../data/endpoint_checker/collection


[{'resource': '07e829ee232d63954874da98e4549befcb8b9a3ea51988799844e9b4a1731def', 'bytes': '39539', 'endpoints': '16857564d1658edeaa7703207b91525d005e7696e2e9392fbb6fa4231fa550ef', 'organisations': 'local-authority-eng:NET', 'datasets': 'article-4-direction-area', 'start-date': '2023-07-17', 'end-date': ''}]


In [ ]:
# validate endpoint


#### collection log summaries

We need to establish if a resource was downloaded from the endpoint and whether there were any issues during the collection process. Examine the output of the below. There should be one log for the attempt made at downloading from the endpoint. If status code is 200 then the resource was downloaded successfully

In [20]:
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
logs = collection.log.entries
logs = pd.DataFrame.from_records(logs)
logs

,bytes,content-type,elapsed,endpoint,resource,status,entry-date,start-date,end-date,exception
0,39539,text/csv,0.513,16857564d1658edeaa7703207b91525d005e7696e2e939...,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,200,2023-07-17T18:28:31.561387,,,


### Check unnassigned entiities
This process automatically aims to detect and assign entity numbers where entries are currently unnassigned. Examine the list below to see what (if any) entities have been assigned. if you were to include these in an actual pipeline you would need to update the configuration lookup.csv with these values. It's worth checking they are sensible before this happens 

In [5]:
unassigned_entries = pd.read_csv(os.path.join(data_dir,'var','cache','unassigned-entries.csv'))
if len(unassigned_entries) == 0:
    print('no additional entity numbers where required')
else:
    unassigned_entries

no additional entity numbers where required


#### Check logs collated from the pipeline process
We need to readin in logs and examine to see if the data points were all read in correctly This uses the sqlite database to do so with some custom queries I have wrote. you could directly examine the csvs if the pipeline fails.

first check column mappings to see what columns the pipeline automatically mapped. if this is empty or missing values it's likely to be the reason data isn't appearing at the end.

In [3]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_column_mappings()
results

,end_date,entry_date,field,dataset,start_date,resource,column
0,,2023-07-17T19:08:28Z,address-text,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,address-text
1,,2023-07-17T19:08:28Z,article-4-direction,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,article-4-direction
2,,2023-07-17T19:08:28Z,article-4-direction-rules,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,article-4-direction-rules
3,,2023-07-17T19:08:28Z,end-date,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,end-date
4,,2023-07-17T19:08:28Z,entry-date,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,entry-date
5,,2023-07-17T19:08:28Z,geometry,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,geometry
6,,2023-07-17T19:08:28Z,name,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,name
7,,2023-07-17T19:08:28Z,notes,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,notes
8,,2023-07-17T19:08:28Z,reference,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,reference
9,,2023-07-17T19:08:28Z,start-date,article-4-direction-area,,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,start-date


examine the issues logs, we'll look at the types of errors being raised and list all of them. This could be improved in the future by examining the severity

In [6]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues_by_type()
results

,issue_type,count
0,default-value,13
1,invalid geometry,1


In [5]:
results = dataset_db.get_issues()
results

,end_date,entry_date,entry_number,field,issue_type,line_number,dataset,resource,start_date,value
0,,,1,organisation,default-value,2,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
1,,,2,organisation,default-value,3,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
2,,,3,geometry,invalid geometry,4,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,Too few points in geometry component[-1.609947...
3,,,3,organisation,default-value,4,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
4,,,4,organisation,default-value,5,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
5,,,5,organisation,default-value,6,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
6,,,6,organisation,default-value,7,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
7,,,7,organisation,default-value,8,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
8,,,8,organisation,default-value,9,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET
9,,,9,organisation,default-value,10,article-4-direction-area,07e829ee232d63954874da98e4549befcb8b9a3ea51988...,,local-authority-eng:NET


#### Final dataset comparison against the sqlite database

Below are two tables which show the difference betwen what was provided to us and what is ucrrently in the entity table. It is important to bear in mind that we assign entities automaticallyis process, the table above shows what we have added.

In [4]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_entities()
results

,dataset,end_date,entity,entry_date,geojson,geometry,json,name,organisation_entity,point,prefix,reference,start_date,typology
0,article-4-direction-area,,7010002370,2023-02-17,,"MULTIPOLYGON (((-1.620664 54.970388,-1.620538 ...","{""article-4-direction"": ""A4BV"", ""article-4-dir...",Brandy Vaults,228,POINT(-1.620648 54.970455),article-4-direction-area,A4BV,1991-07-18,geography
1,article-4-direction-area,,7010002371,2023-02-17,,"MULTIPOLYGON (((-1.743241 54.983214,-1.743166 ...","{""article-4-direction"": ""A4DC"", ""article-4-dir...",Dukes Cottages,228,POINT(-1.742842 54.983072),article-4-direction-area,A4DC,1983-09-14,geography
2,article-4-direction-area,,7010002372,2023-02-17,,"MULTIPOLYGON (((-1.633350 54.983025,-1.633184 ...","{""article-4-direction"": ""A4HMO1"", ""article-4-d...",Jesmond and Heaton,228,POINT(-1.597894 54.987950),article-4-direction-area,A4HMO1,2011-11-25,geography
3,article-4-direction-area,,7010002373,2023-02-17,,"MULTIPOLYGON (((-1.611846 54.997747,-1.611389 ...","{""article-4-direction"": ""A4HMO2"", ""article-4-d...",HW Jesmond & N Jesmond,228,POINT(-1.608322 54.998922),article-4-direction-area,A4HMO2,2012-12-09,geography
4,article-4-direction-area,,7010002374,2023-02-17,,"MULTIPOLYGON (((-1.583702 54.991184,-1.586195 ...","{""article-4-direction"": ""A4HMO3"", ""article-4-d...","Saint Gabriel's Estate, Heaton",228,POINT(-1.586163 54.986095),article-4-direction-area,A4HMO3,2012-08-09,geography
5,article-4-direction-area,,7010002375,2023-02-17,,"MULTIPOLYGON (((-1.728462 54.999055,-1.728520 ...","{""article-4-direction"": ""A4NG"", ""article-4-dir...",Northumberland Gardens,228,POINT(-1.725979 54.999727),article-4-direction-area,A4NG,1997-03-19,geography
6,article-4-direction-area,,7010002376,2023-02-17,,"MULTIPOLYGON (((-1.726733 55.000051,-1.726785 ...","{""article-4-direction"": ""A4NG"", ""article-4-dir...",Northumberland Gardens Exception,228,POINT(-1.726031 54.999927),article-4-direction-area,A4NG/E,1997-03-19,geography
7,article-4-direction-area,,7010002377,2023-02-17,,"MULTIPOLYGON (((-1.625810 54.969804,-1.625196 ...","{""article-4-direction"": ""A4SH"", ""article-4-dir...",Summerhill,228,POINT(-1.627090 54.970420),article-4-direction-area,A4SH,1990-12-19,geography
8,article-4-direction-area,,7010002378,2023-02-17,,"MULTIPOLYGON (((-1.576093 54.965816,-1.576198 ...","{""article-4-direction"": ""A4SPB"", ""article-4-di...",Saint Peter's Basin 1,228,POINT(-1.575287 54.965476),article-4-direction-area,A4SPBa1,1996-07-13,geography
9,article-4-direction-area,,7010002379,2023-02-17,,"MULTIPOLYGON (((-1.575562 54.966857,-1.575579 ...","{""article-4-direction"": ""A4SPB"", ""article-4-di...",Saint Peter's Basin 2,228,POINT(-1.574809 54.966515),article-4-direction-area,A4SPBa2,1996-07-14,geography


In [7]:
# load in raw resources
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
resources = collection.resource.entries
resources

[{'resource': '07e829ee232d63954874da98e4549befcb8b9a3ea51988799844e9b4a1731def',
  'bytes': '39539',
  'organisations': 'local-authority-eng:NET',
  'datasets': 'article-4-direction-area',
  'endpoints': '16857564d1658edeaa7703207b91525d005e7696e2e9392fbb6fa4231fa550ef',
  'start-date': '2023-07-17',
  'end-date': ''}]

In [9]:
converted_resource = pd.read_csv(os.path.join(data_dir,'collection','resource',resources[0]['resource']))
converted_resource

,reference,name,geometry,address-text,article-4-direction,article-4-direction-rules,notes,start-date,end-date,entry-date
0,A4BV,Brandy Vaults,"MultiPolygon (((-1.62066432 54.97038806, -1.62...",NaN,A4BV,P2CC,The exterior of the building can not be painte...,1991-07-18,NaN,2023-02-17
1,A4DC,Dukes Cottages,"MultiPolygon (((-1.74324121 54.9832141, -1.743...",NaN,A4DC,P1CA;P1CB;P1CC;P1CD;P1CE;P1CF;P1CH;P2CA;P2CB;P2CC,No enlargements or alterations shall not be ca...,1983-09-14,NaN,2023-02-17
2,A4HMO1,Jesmond and Heaton,"MultiPolygon (((-1.609947 55.003795, -1.609946...",NaN,A4HMO1,P3CL,"In certain areas of Newcastle, planning permis...",2011-11-25,NaN,2023-02-17
3,A4HMO2,HW Jesmond & N Jesmond,"MultiPolygon (((-1.61184578 54.99774668, -1.61...",NaN,A4HMO2,P3CL,"In certain areas of Newcastle, planning permis...",2012-12-09,NaN,2023-02-17
4,A4HMO3,"Saint Gabriel's Estate, Heaton","MultiPolygon (((-1.58370217 54.99118367, -1.58...",NaN,A4HMO3,P3CL,"In certain areas of Newcastle, planning permis...",2012-08-09,NaN,2023-02-17
5,A4NG,Northumberland Gardens,"MultiPolygon (((-1.72846204 54.99905527, -1.72...",NaN,A4NG,P1CA;P1CC;P1CD;P1CE;P2CA;P2CC,A number of changes to properties within this ...,1997-03-19,NaN,2023-02-17
6,A4NG/E,Northumberland Gardens Exception,"MultiPolygon (((-1.72673262 55.00005058, -1.72...",NaN,A4NG,P1CA;P1CD;P1CE;P2CA;P2CC,A number of changes to properties within this ...,1997-03-19,NaN,2023-02-17
7,A4SH,Summerhill,"MultiPolygon (((-1.62581025 54.96980438, -1.62...",NaN,A4SH,P1CA;P1CC;P1CD;P1CE;P1CF;P1CH;P2CA;P2CB;P2CC,A number of changes to properties within this ...,1990-12-19,NaN,2023-02-17
8,A4SPBa1,Saint Peter's Basin 1,"MultiPolygon (((-1.57609269 54.96581624, -1.57...",NaN,A4SPB,P1CA;P1CB;P1CC;P1CD;P1CH;P2CA;,Enlargements and alterations of properties at ...,1996-07-13,NaN,2023-02-17
9,A4SPBa2,Saint Peter's Basin 2,"MultiPolygon (((-1.57556216 54.96685748, -1.57...",NaN,A4SPB,P1CA;P1CB;P1CC;P1CD;P1CH;P2CA;P2CC,Enlargements and alterations of properties at ...,1996-07-14,NaN,2023-02-17
